# Cleaning and Preprocessing the Monthly Webinar Data for Making a Standard Structure

This notebook is intended to analyze the data of the monthly webinars in the MEXICAN AESCULAP ACADEMY.

Author : Axel Daniel Malváez Flores  
Date : July 13th, 2023  
Version : 1.0.0  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

In [2]:
data = pd.read_excel('data.xlsx', sheet_name='data_structured')
data.head()

,respondent_id,collector_id,date_created,date_modified,ip_address,email_address,first_name,last_name,custom_1,Abreviación de su titulo: (Response),...,Día Internacional del Sarcoma,Organización general:,Modalidad virtual (Zoom):,Horario:,¿Cómo calificaría esta sesión en general?,¿Qué es lo que más te gusta?,¿Qué otros temas te interesarían?,¿Qué podriamos mejorar en los eventos de Academia Aesculap?,¿Qué tan probable es que recomiendes está sesión a tus amigos o familiares?,Aviso de Privacidad
0,114365999487,426790957,2023-07-12 23:31:35,2023-07-12 23:33:45,187.190.207.107,NaN,NaN,NaN,NaN,Mtra.,...,Bueno,Bueno,Bueno,Bueno,Bueno,Gracias academia aesculap,Código rojo,NaN,10,He leído y estoy de acuerdo con el AVISO DE P...
1,114365979045,426790957,2023-07-12 22:36:58,2023-07-12 22:41:13,189.216.207.34,NaN,NaN,NaN,NaN,Dr.,...,Bueno,Bueno,Bueno,Bueno,8,Contenido organización y buen nivel,Ortogeriatria,NaN,10,He leído y estoy de acuerdo con el AVISO DE P...
2,114365975998,426790957,2023-07-12 22:28:30,2023-07-12 22:33:27,201.175.240.109,NaN,NaN,NaN,NaN,Dra.,...,Excelente,Excelente,Excelente,Excelente,Excelente,La interacción que tienen los médicos para com...,"Lupus eritematoso, glomerulopatias",NaN,10,He leído y estoy de acuerdo con el AVISO DE P...
3,114365969665,426790957,2023-07-12 21:42:48,2023-07-12 22:17:30,189.234.132.175,NaN,NaN,NaN,NaN,Otro (especifique),...,Bueno,Bueno,Bueno,Bueno,8,Todo estubo bien,Por el momento ninguno,El audio,8,He leído y estoy de acuerdo con el AVISO DE P...
4,114365965316,426790957,2023-07-12 20:13:10,2023-07-12 22:07:11,189.203.52.31,NaN,NaN,NaN,NaN,Otro (especifique),...,Excelente,Excelente,Excelente,Excelente,Excelente,Todo,Lavado Quirúrgico,NaN,10,He leído y estoy de acuerdo con el AVISO DE P...


* Extracting the columns that doesn't contain 'Otro (especifique)'

In [3]:
import re
regex = re.compile(r'Otro \(especifique\)')
# if the column contains the regex, don't extract it
valid_cols = []
for col in data.columns:
    if regex.search(col):
        continue
    else:
        valid_cols.append(col)

data_valid = data[valid_cols]
data_others = data[[col for col in data.columns if col not in valid_cols]]

## Data Profiling

In [4]:
# pandas profiling
import pandas_profiling as pp

# html = pp.ProfileReport(data_valid)
# html.to_file(output_file="output.html")

/var/folders/8l/f787_tgj7sg6grwn27dvfx0h0000gn/T/ipykernel_55549/3848066059.py:2: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling as pp


* Checking whether *data_created* and *data_modified* have the same values

In [7]:
data.loc[:,['date_created', 'date_modified']]

,date_created,date_modified
0,2023-07-12 23:31:35,2023-07-12 23:33:45
1,2023-07-12 22:36:58,2023-07-12 22:41:13
2,2023-07-12 22:28:30,2023-07-12 22:33:27
3,2023-07-12 21:42:48,2023-07-12 22:17:30
4,2023-07-12 20:13:10,2023-07-12 22:07:11
...,...,...
204,2023-07-12 20:05:08,2023-07-12 20:08:20
205,2023-07-12 19:55:12,2023-07-12 20:03:34
206,2023-07-12 19:24:16,2023-07-12 19:30:48
207,2023-07-12 19:23:45,2023-07-12 19:27:15


$\therefore$ These columns are not the same, nevertheless *data_modified* is really useful to make an analysis to determine which hours are the most common to answer the survey. Since *date_created* is just the date when the survey was created, it is not useful for this analysis. Consequently, *date_modified* will be used, given that is the column that registers the **timestamp** when the survey was started answered.

In [10]:
# This will be analyzed in the other notebook
hours_responding_survey = data.loc[:,['date_modified']]

* Does all the devices are different?

In [14]:
ip_address_different = len(data.loc[:,'ip_address'].drop_duplicates())
print(f'The total different devices were {ip_address_different} and the total of responses were {len(data)}')

The total different devices were 191 and the total of responses were 209


In [40]:
ip_duplicated = data[data['ip_address'].duplicated(keep=False)].sort_values(by='ip_address').loc[:,['ip_address','Nombre completo:', 'Correo electrónico:']]
ip_duplicated.head()

,ip_address,Nombre completo:,Correo electrónico:
35,177.231.27.236,LILIAN ROMERO ROMERO,lilian072020@hotmail.com
67,177.231.27.236,JOSE BARDESI MANI,bardesi500@hotmail.com
126,177.247.115.170,Claudia Gabriela Miranda Martinez,freyagreypool@gmail.com
177,177.247.115.170,Ma. Concepción Miranda Martínez,connymar1@yahoo.com
81,177.247.115.170,MA. CONCEPCIÓN MIRANDA MARTÍNEZ,connymar1@yahoo.com


Here we need to make a record linkage to verify if there exist some repeated participants in the survey.

* Verify if the columns *email_address*, *first_name*, *last_name* and *custom_1* contains any single value

In [43]:
data.loc[:,['email_address', 'first_name', 'last_name', 'custom_1']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   email_address  0 non-null      float64
 1   first_name     0 non-null      float64
 2   last_name      0 non-null      float64
 3   custom_1       0 non-null      float64
dtypes: float64(4)
memory usage: 6.7 KB


No, so we can delete them from the dataset.

In [44]:
data = data.drop(columns=['email_address', 'first_name', 'last_name', 'custom_1'])

* ¿Qué podriamos mejorar en los eventos de Academia Aesculap?

In [46]:
data['¿Qué podriamos mejorar en los eventos de Academia Aesculap?'].value_counts()

El audio    9
Name: ¿Qué podriamos mejorar en los eventos de Academia Aesculap?, dtype: int64

## Data Cleaning